In [27]:
f = open('data.txt','r')
readfile = f.read().split('\n')
n, m, k = map(int,readfile[0].split())
tmp_q = map(int, readfile[1].split())
Q = list(map(int, readfile[2].split()))
d = list()
# for i in range(k):
for i in range(3,3 + 2*n + 2*m + 1):
    d.append(list(map(int,readfile[i].split())))
    # print(*map(int,readfile[i].split()))
########  thừa
# I = [i for i in range(1,n+1)] # tập điểm nhận khách
# L = [i for i in range(n+1,n+m+1)] # tập điểm nhận hàng
# M = [i for i in range(n+m+1, 2*n + 2*m + 1)] # tập điểm trả khách và hàng
########
q = [] # khối lượng hàng hóa
t = [] # số lượng người
for i in range(k):
    q.append(0)
    t.append(0)
for i in range(k, k + n):
    q.append(0)
    # if(i == 0):
    #     t.append(0)
    # else:
    t.append(1)
for i in tmp_q:
    q.append(i)
    t.append(0)
for i in range(k+ n + m , k + 2*n + 2*m):
    q.append(-q[i-n-m])
    t.append(-t[i-n-m])
P_ = [i for i in range(2*n + 2*m + k)] 
# print(P_)
for i in range(k-1):
    (d[0]).insert(0,0)

ep = d[0]
for i in range(k-1):
    d.insert(0, ep)

for i in range(k, k + 2*n + 2*m ):
    tmp = d[i][0]
    d[i].insert(0, tmp)
    d[i].insert(0, tmp)


for i in range(2*n + 2*m + k):
    print(*d[i])
# print(q)
# print(t)

0 0 0 8 3 3 6 4 4 8 9
0 0 0 8 3 3 6 4 4 8 9
0 0 0 8 3 3 6 4 4 8 9
3 3 3 0 7 8 1 4 2 2 8
9 9 9 8 0 6 7 3 5 8 1
7 7 7 5 2 0 5 7 4 3 7
8 8 8 3 3 8 0 8 2 2 1
6 6 6 7 9 2 5 0 4 3 2
8 8 8 5 5 7 4 7 0 7 2
1 1 1 6 8 1 7 8 1 0 3
8 8 8 7 2 7 9 2 8 5 0


In [43]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

###########variables################
R = [model.NewIntVar(0, 2*n + 2*m + k - 1 , f'R[{i}]') for i in range(2*n + 2*m + k)] # vector điểm tiếp theo được đi đến
###################################
V = list()
D= list()
w = list()
T = list()

for i in range(2*n + 2*m + k): ########## Có vấn đề khi khởi tạo V D w T
    V.append(0)
    D.append(0)
    w.append(0)
    T.append(0)

###########constraints################
model.AddAllDifferent(R)
for i in range(2*n + 2*m + k):
    model.Add(R[i] >= 0)
    model.Add(R[i] <= 2*n + 2*m + k -1)
model.Add(R[10] == 0)
for i in range(k):
    model.Add(D[i] == 0)
    model.Add(w[i] == i)
    model.Add(V[i] == 0)
    model.Add(T[i] == 0)

for i in range(1, 2*n + 2*m + k):
    b = model.NewBoolVar('b')
    target_element_V =  model.NewIntVar(0, 100, 'tV')
    target_element_T =  model.NewIntVar(0, 1, 'tT')
    target_element_D =  model.NewIntVar(0, 100, 'tD')
    target_element_w =  model.NewIntVar(0, k-1, 'tw')
    target_element_d =  model.NewIntVar(0, 100, 'td')
    model.AddElement(R[i], V, target_element_V)
    model.AddElement(R[i], T, target_element_T)
    model.AddElement(R[i], D, target_element_D)
    model.AddElement(R[i], w, target_element_w)
    model.AddElement(R[i], d[i], target_element_d)

    model.Add(R[i] >= k).OnlyEnforceIf(b)
    model.Add(R[i] < k).OnlyEnforceIf(b.Not())
    model.Add(target_element_V == V[i] + q[i]).OnlyEnforceIf(b) ######
    model.Add(target_element_V == 0).OnlyEnforceIf(b.Not())
    model.Add(target_element_T == T[i] + t[i]).OnlyEnforceIf(b)
    model.Add(target_element_T == 0).OnlyEnforceIf(b.Not())
    model.Add(target_element_D == D[i] + target_element_d).OnlyEnforceIf(b)
    model.Add(target_element_D == 0).OnlyEnforceIf(b.Not())
    model.Add(target_element_w == w[i]).OnlyEnforceIf(b)
    model.Add(target_element_w == R[i]).OnlyEnforceIf(b.Not())

for i in range(k, n + m + k):
    model.Add(w[i] == w[i + n + m])
    model.Add(D[i] <= D[i + n + m])

for i in range(2*n + 2*m + k):
    # target_element_Q =  model.NewIntVar(0, 1000, 'tQ')
    # model.AddElement(w[i], Q, target_element_Q)
    target_element_Q = Q[w[i]]
    model.Add(0 <= T[i])
    model.Add(T[i] <= 1)
    model.Add(V[i] <= target_element_Q)

max_D = model.NewIntVar(0, 100 ,'max_D')
for i in range(2*n + 2*m + k):
    model.Add(max_D >= D[i])
model.Minimize(max_D)

solver = cp_model.CpSolver()

status = solver.Solve(model)
print("Objective value ",solver.Value(max_D))
# if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
#     print("Objective value ",solver.Value(max_D))
# else:
#     print('No solution found.')

IndexError: list index out of range